### Introduction
- We will explore how to ingest JSON files and perform foundational JSON speciifc transformations during ingestion
- We will use simulated Kafka event data

### Learning Objectives
- Ingest raw JSON into Unity Catalog using CTAS and read_files()
- Apply techniques to flatten JSON string columns
- Understand difference between explode() and explode_outer()
- Introduce capabilities and use cases of VARAINT data type

### Setup
- run 

In [0]:
%run ../01_Data_Engineer_Learning_Plan/Lab-Setup/lab-setup-05

### Explore the data
- There should be 6 keys.
- key and value fields are encoded in base64.(Encoding scheme that converts binary data into a readable ASCII string)

In [0]:
-- View current catalog and schema you are using
SELECT current_catalog(), current_schema()

In [0]:
-- Verify that there are 11 JSON files in our volume
LIST '/Volumes/workspace/data_engineering_labs_00/v01/raw/events-kafka/'

In [0]:
-- Using text helps us get a quick preview
SELECT * 
FROM text.`/Volumes/workspace/data_engineering_labs_00/v01/raw/events-kafka/`
LIMIT 5;

### 1. Read the files into tablular format using read_files()
- Lets store the raw data into a table called `kafka_events_bronze_raw`
- Then lets decode them into a second layer bronze table `kafka_events_bronze_decoded` (We could always just have 1 layer, its up to you)

In [0]:
-- Explore how it looks like in Tabular form
SELECT * 
FROM read_files(
  "/Volumes/workspace/data_engineering_labs_00/v01/raw/events-kafka/",
  format => "json"
)
LIMIT 10;

In [0]:
-- DROP TABLE IF Exists
DROP TABLE IF EXISTS workspace.data_engineering_labs_00.kafka_events_bronze_raw;

-- Create the delta bronze table
CREATE TABLE data_engineering_labs_00.kafka_events_bronze_raw AS
SELECT * 
FROM read_files(
  "/Volumes/workspace/data_engineering_labs_00/v01/raw/events-kafka/",
  format => "json"
);

-- Display
SELECT * FROM data_engineering_labs_00.kafka_events_bronze_raw
LIMIT 10

Decode base64 strings for the bronze table
- Use `unbase64()` function
- Note that the decoded values are now binary


In [0]:
SELECT 
key as encoded_key,
unbase64(key) as decoded_key,
value as encoded_value,
unbase64(value) as decoded_value
FROM data_engineering_labs_00.kafka_events_bronze_raw
LIMIT 5;

- We now use `cast()` to convert binary columns to Strings, making them type strign and readable
- decoded value column ius now a JSON -formatted string

In [0]:
SELECT 
key as encoded_key,
cast(unbase64(key) as string) as decoded_key,
value as encoded_value,
cast(unbase64(value)as string ) as decoded_value
FROM data_engineering_labs_00.kafka_events_bronze_raw
LIMIT 5;

- Now we put them all tgt and create the 2nd level bronze table `kafka_events_bronze_decoded`
- ignore if decoded key look strange

In [0]:
CREATE OR REPLACE TABLE data_engineering_labs_00.kafka_events_bronze_decoded AS 
SELECT 
  cast(unbase64(key) as string) as decoded_key,
  offset,
  partition,
  timestamp,
  topic,
  cast(unbase64(value)as string ) as decoded_value
FROM data_engineering_labs_00.kafka_events_bronze_raw;

--View
SELECT * FROM data_engineering_labs_00.kafka_events_bronze_decoded
LIMIT 5

### 2a. Working with JSON-formatted strings in a table (flatten it using `:`)

Benefits
- Simple: Easy to implment and store JSON as plain text
- Flexible: Can hold any JSON structure without schema constraints

Considerations:
- Performance: STRING columns are slower when querying and porocesing data
- No Schema: Lack of a defined schema for STRING column can lead to data integrity issus
- Complex to query: Requitres additional code to parse and retrieve data

How to:
 Query using `column_name : extraction_path` using `.` or `[]` when accessing nested data

In [0]:
SELECT 
decoded_value:device,
decoded_value:traffic_source,
decoded_value:geo, --contains anotheer JSON formatted string
decoded_value:items --contains a nested array of JSON fomatted strings
FROM data_engineering_labs_00.kafka_events_bronze_decoded
LIMIT 5

Create the Table `kafka_events_bronze_string_flattened`

In [0]:
CREATE OR REPLACE TABLE data_engineering_labs_00.kafka_events_bronze_string_flattened AS 
SELECT
decoded_key,
offset,
partition,
timestamp,
topic,
decoded_value:device,
decoded_value:traffic_source,
decoded_value:geo, --contains anotheer JSON formatted string
decoded_value:items --contains a nested array of JSON fomatted strings

FROM data_engineering_labs_00.kafka_events_bronze_decoded;

--View
SELECT * FROM data_engineering_labs_00.kafka_events_bronze_string_flattened
LIMIT 5

### 2b. Working with JSON-formatted strings in a table (flatten it using STRUCT conversion)

Benefits
- Schema enforcement: STRUCT columns dfin and enforce a schema, helping maintain data integirty
-Improved Performanc = STRUCTs are more efficient for querying and procssing then strings

Considerations
- Schema enforcment: As schema is enforced, issues may arise if JSON structure change over timee
- Reduced flexibility: Data must consistentlky match the defined schema, less room for structural variation

How to: (2 steps)
1. Get schema of string using `schema_of_json`, paste an example schema there and it will help us retrieve it as type STRUCT
2. Apply schema to json formatted column by copy and paste the output from step 1 into `from_json()` function


In [0]:
-- This will return the schema as a type STRUCT
SELECT schema_of_json(
'{"device":"Windows","ecommerce":{"purchase_revenue_in_usd":1195.0,"total_item_quantity":1,"unique_items":1},"event_name":"main","event_timestamp":176757862507185181,"geo":{"city":"San Francisco","state":"CA"},"items":[{"coupon":null,"item_id":"M_STAN_K","item_name":"Standard King Mattress","item_revenue_in_usd":1195.0,"price_in_usd":1195.0,"quantity":1}],"traffic_source":"google","user_first_touch_timestamp":1762394625071,"user_id":"UA1043321819"}')
AS SCHEMA

In [0]:
-- Final table
CREATE OR REPLACE TABLE data_engineering_labs_00.kafka_events_bronze_string_struct AS 
SELECT * EXCEPT (decoded_value),
from_json(decoded_value, 
'STRUCT<device: STRING, ecommerce: STRUCT<purchase_revenue_in_usd: DOUBLE, total_item_quantity: BIGINT, unique_items: BIGINT>, event_name: STRING, event_timestamp: BIGINT, geo: STRUCT<city: STRING, state: STRING>, items: ARRAY<STRUCT<coupon: STRING, item_id: STRING, item_name: STRING, item_revenue_in_usd: DOUBLE, price_in_usd: DOUBLE, quantity: BIGINT>>, traffic_source: STRING, user_first_touch_timestamp: BIGINT, user_id: STRING>'
) AS Value

FROM data_engineering_labs_00.kafka_events_bronze_decoded;

--View
SELECT * FROM data_engineering_labs_00.kafka_events_bronze_string_struct
LIMIT 5

- We can then use `.` to query the STRUCT column

In [0]:
SELECT decoded_key,
value.device as dvice,
value.geo.city as city,
value.items as items,
array_size(items) as number_of_elemnts_in_array
FROM data_engineering_labs_00.kafka_events_bronze_string_struct
ORDER BY number_of_elemnts_in_array DESC
LIMIT 5

### 3. Note the items columns
  - It is an array of Elments.
  - We can use `explode` to transofrm each element into its own sparate row.
  - If NULL, no rows are produced. If we want to return rows when there are nulls, use `explode_outer()`

In [0]:
CREATE OR REPLACE TABLE data_engineering_labs_00.kafka_events_bronze_explode_array AS 
SELECT decoded_key,
array_size(value.items) AS number_elements_in_array,
explode(value.items) as item_in_array, --individual item of the array
value.items --original data for comparision


FROM data_engineering_labs_00.kafka_events_bronze_string_struct
ORDER BY number_elements_in_array DESC;

--View
SELECT * FROM data_engineering_labs_00.kafka_events_bronze_explode_array
LIMIT 10

### 4. Using VARIANT
Beenefits:
- Open source
- Flexible: No strict schma, can put any kind of semi structuree data
- Performant: Improvd prformancw over exisitng methods

How to:
- Use `parse_json` function to return a VARIANT value
- We can pars variant data `:` to create desired table

In [0]:
-- Lets use the decodd table for a start
SELECT *
FROM data_engineering_labs_00.kafka_events_bronze_decoded
LIMIT 5;

Note that after running cell, json_variant_value is of type variant


In [0]:
CREATE OR REPLACE TABLE data_engineering_labs_00.kafka_events_bronze_variant AS 
SELECT 
  decoded_key,
  offset,
  partition,
  timestamp,
  topic,
  parse_json(decoded_value) AS json_variant_value
FROM data_engineering_labs_00.kafka_events_bronze_decoded;

--View
SELECT * FROM data_engineering_labs_00.kafka_events_bronze_variant
LIMIT 10

In [0]:
-- USe : to create desired tabl

SELECT
json_variant_value,
json_variant_value:device :: STRING, -- parse device as string
json_variant_value:items
FROM data_engineering_labs_00.kafka_events_bronze_variant
LIMIT 10